In [1]:
import pandas as pd
import numpy as np
import re
import time
import ast
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline

from sklearn.multiclass import OneVsRestClassifier
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

from sklearn.preprocessing import MultiLabelBinarizer

from sklearn.model_selection import KFold
from sklearn.metrics import log_loss

from gensim import corpora, models, similarities

# Linear Models 
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn import linear_model
from sklearn.naive_bayes import MultinomialNB



C:\Anaconda2\envs\py36\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
nameTrainCSV = 'trainWithListOfCleanWords'
nameTestCSV = 'testWithListOfCleanWords'

train = pd.read_csv('../data/processed/' + nameTrainCSV + '.csv', encoding='utf-8')
train['BagOfWords'] = dict
train.head(10)


,Unnamed: 0,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,listOfCleanWords,cleanWordsAsText,BagOfWords
0,0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,"['explanation', 'edits', 'made', 'username', '...",explanation edits made username hardcore metal...,<class 'dict'>
1,1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,"['aww', 'match', 'background', 'colour', 'seem...",aww match background colour seemingly stuck th...,<class 'dict'>
2,2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,"['hey', 'man', 'really', 'trying', 'edit', 'wa...",hey man really trying edit war guy constantly ...,<class 'dict'>
3,3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,"['make', 'real', 'suggestion', 'improvement', ...",make real suggestion improvement wondered sect...,<class 'dict'>
4,4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,"['sir', 'hero', 'chance', 'remember', 'page']",sir hero chance remember page,<class 'dict'>
5,5,00025465d4725e87,"""\n\nCongratulations from me as well, use the ...",0,0,0,0,0,0,"['congratulation', 'well', 'use', 'tool', 'wel...",congratulation well use tool well talk,<class 'dict'>
6,6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0,"['cocksucker', 'piss', 'around', 'work']",cocksucker piss around work,<class 'dict'>
7,7,00031b1e95af7921,Your vandalism to the Matt Shirvington article...,0,0,0,0,0,0,"['vandalism', 'matt', 'shirvington', 'article'...",vandalism matt shirvington article reverted pl...,<class 'dict'>
8,8,00037261f536c51d,Sorry if the word 'nonsense' was offensive to ...,0,0,0,0,0,0,"['sorry', 'word', 'nonsense', 'offensive', 'an...",sorry word nonsense offensive anyway intending...,<class 'dict'>
9,9,00040093b2687caa,alignment on this subject and which are contra...,0,0,0,0,0,0,"['alignment', 'subject', 'contrary', 'dulithgow']",alignment subject contrary dulithgow,<class 'dict'>


In [3]:
start = time.time()
for x in range(len(train)):
    train.set_value(col='listOfCleanWords',
                index=x,
                value=ast.literal_eval(train["listOfCleanWords"][x]))
    train.set_value(col='BagOfWords',
                index=x,
                value=Counter(train["listOfCleanWords"][x]))
end = time.time()
print(end - start)

16.68869686126709


In [4]:
train.head()

,Unnamed: 0,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,listOfCleanWords,cleanWordsAsText,BagOfWords
0,0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,"[explanation, edits, made, username, hardcore,...",explanation edits made username hardcore metal...,"{'explanation': 1, 'edits': 1, 'made': 1, 'use..."
1,1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,"[aww, match, background, colour, seemingly, st...",aww match background colour seemingly stuck th...,"{'aww': 1, 'match': 1, 'background': 1, 'colou..."
2,2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,"[hey, man, really, trying, edit, war, guy, con...",hey man really trying edit war guy constantly ...,"{'hey': 1, 'man': 1, 'really': 1, 'trying': 1,..."
3,3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,"[make, real, suggestion, improvement, wondered...",make real suggestion improvement wondered sect...,"{'make': 1, 'real': 1, 'suggestion': 1, 'impro..."
4,4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,"[sir, hero, chance, remember, page]",sir hero chance remember page,"{'sir': 1, 'hero': 1, 'chance': 1, 'remember':..."


In [5]:
# MULTICLASS PREDICTION

In [6]:
classLabel = {
    0: "neutral",
    1: "toxic",
    2 : "severe_toxic",
    3 : "obscene",
    4 : "threat",
    5 : "insult",
    6 : "identity_hate" 
}


In [7]:
y = np.empty((len(train['cleanWordsAsText']),),dtype=object)
allTextToxicTrain = dict()
for idx in classLabel:
    if classLabel[idx] != "neutral":
        T = np.where(train[classLabel[idx]] == 1)[0]
        allTextToxicTrain[idx] = T
        for i in T:
            if y[i] is None:
                y[i] = [idx]                
            else:
                y[i].append(idx)
indxsOfNeutralTexts = np.where(y == None) 
y[indxsOfNeutralTexts] = [[0]]
indxsOfNeutralTexts = indxsOfNeutralTexts[0]

allTextsNoToxicTrain = [str(train['cleanWordsAsText'][x]) for x in indxsOfNeutralTexts]

idxList = []
for i in allTextToxicTrain.keys():
    #allTextToxicTrain[i] = [str(train['cleanWordsAsText'][j]) for j in allTextToxicTrain[i]]
    idxList = np.unique(np.append(idxList, allTextToxicTrain[i]))
allTextToxicTrain = [str(train['cleanWordsAsText'][j]) for j in idxList]

In [8]:
columns = ["idExp","numFeatures", "algorithm", "Nfolds", "accuaracy", "logloss", "fmeasure"]
dfTestResults = pd.DataFrame(columns=columns)


In [9]:
listTopNFeatures = [10,50,100,200,500,1000]

In [10]:
# Test clasification
test = pd.read_csv('../data/processed/' + nameTestCSV + '.csv', encoding='utf-8')
test.head()


,Unnamed: 0,id,comment_text,listOfCleanWords,cleanWordsAsText
0,0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...,"['yo', 'bitch', 'ja', 'rule', 'succesful', 'ev...",yo bitch ja rule succesful ever whats hating s...
1,1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...,"['rfc', 'title', 'fine', 'imo']",rfc title fine imo
2,2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap...","['source', 'zawe', 'ashton', 'lapland']",source zawe ashton lapland
3,3,00017563c3f7919a,":If you have a look back at the source, the in...","['look', 'back', 'source', 'information', 'upd...",look back source information updated correct f...
4,4,00017695ad8997eb,I don't anonymously edit articles at all.,"['anonymously', 'edit', 'article']",anonymously edit article


In [11]:
# PREDICCION MULTICLASS TEST
topNFeatures = listTopNFeatures[3]
#Neutral data

tdidfVectTrainNoToxic = TfidfVectorizer(max_features=topNFeatures)
matrixTdidfTrainNoToxic = tdidfVectTrainNoToxic.fit_transform(allTextsNoToxicTrain)

featureArrayNoToxicTrain = np.array(tdidfVectTrainNoToxic.get_feature_names())
tfidfOrderedNoToxicTrain = np.argsort(matrixTdidfTrainNoToxic.toarray()).flatten()[::-1]
selectedWordsTrainNoToxic = featureArrayNoToxicTrain[tfidfOrderedNoToxicTrain][:topNFeatures]

#Toxic data
selectedWordsTrainToxic = []
selectedWordsTrain = []

tdidfVectTrainToxic = TfidfVectorizer(max_features=topNFeatures)
matrixTdidfTrainToxic = tdidfVectTrainToxic.fit_transform(allTextToxicTrain)
featureArrayToxicTrain = np.array(tdidfVectTrainToxic.get_feature_names())
tfidfOrderedToxicTrain = np.argsort(matrixTdidfTrainToxic.toarray()).flatten()[::-1]
selectedWordsTrainToxic = featureArrayToxicTrain[tfidfOrderedToxicTrain][:topNFeatures]

# Get words
for x in range(topNFeatures):
    if len(selectedWordsTrain) <= topNFeatures: 
        words = [selectedWordsTrainToxic[x]] + [selectedWordsTrainNoToxic[x]]
        selectedWordsTrain = np.unique(np.append(selectedWordsTrain, np.unique(words)))    
selectedWordsTrain = selectedWordsTrain[:topNFeatures]

allTrainText = [txt if txt is not np.nan else '' for txt in train['cleanWordsAsText']]
allTestText = [txt if txt is not np.nan else '' for txt in test['cleanWordsAsText']]
X_train = allTrainText
X_test = allTestText
yBinary = MultiLabelBinarizer().fit_transform(y)
y_train = yBinary


vectTrain = CountVectorizer(vocabulary=selectedWordsTrain)
X_train_bow = vectTrain.fit_transform(X_train).toarray()
X_test_bow = vectTrain.fit_transform(X_test).toarray()


In [12]:
# Word 2 Vec mean train list
modelWord2VecTrain = models.Word2Vec(train['listOfCleanWords'].tolist(), min_count=1)
#say_vector = modelWord2VecTrain['explanation']  # get vector for word
#say_vector

In [13]:
vectorizedTrainW2V = np.empty(len(train['listOfCleanWords']), dtype=list)
for i, lstOfCleanWords in enumerate(train['listOfCleanWords']): 
    size = len(lstOfCleanWords)
    if size > 0:
        vector = np.zeros(len(modelWord2VecTrain[lstOfCleanWords[0]]))
        for x in range(size):
            vector += modelWord2VecTrain[lstOfCleanWords[x]]
        vectorizedTrainW2V[i] = vector / size

C:\Anaconda2\envs\py36\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """
C:\Anaconda2\envs\py36\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  import sys


In [14]:
vectorizedTrainW2V

array([ array([ 0.73379291, -0.15204202,  0.16799682, -0.09350665, -0.87342633,
       -0.12644328, -0.29345003, -0.47284597,  0.94774098, -0.33401104,
       -0.35927565,  1.13237797, -0.07214655,  0.32651917,  0.19017308,
        0.17825947, -0.21004198, -0.73069149,  0.64441242, -0.40265178,
        1.02520506, -0.47286715,  1.48720414,  0.57169645,  0.42970759,
       -0.08890211,  0.63591857,  0.10385049,  0.7023698 , -0.67173565,
       -0.74867027,  0.9034874 , -0.13497437, -0.17129793, -0.16749973,
       -0.04919034, -0.24335117,  0.13657868,  0.67659103, -0.10070693,
       -0.30201212,  0.31276183,  0.22123449, -0.34346243, -0.49338947,
       -0.61131901, -0.94420535, -0.19631918,  0.4732139 ,  0.28291244,
        0.75220588,  0.44198064,  0.25020587, -0.0982263 ,  0.50892403,
        0.17714172,  0.0810034 , -0.16032361,  0.42767846,  0.08251705,
        0.4427396 ,  0.27537667, -1.17818363, -0.20494232,  0.48081415,
        0.85164736,  0.082337  , -0.19297625,  0.0564974

In [15]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import keras
from keras import optimizers
from keras import backend as K
from keras import regularizers
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten
from keras.layers import Embedding, Conv1D, MaxPooling1D, GlobalMaxPooling1D 
from keras.utils import plot_model
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping

from tqdm import tqdm
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer 
import os, re, csv, math, codecs

sns.set_style("whitegrid")
np.random.seed(0)

DATA_PATH = '../input/'
EMBEDDING_DIR = '../input/'

MAX_NB_WORDS = 100000
tokenizer = RegexpTokenizer(r'\w+')
stop_words = set(stopwords.words('english'))
stop_words.update(['.', ',', '"', "'", ':', ';', '(', ')', '[', ']', '{', '}'])

from subprocess import check_output


Using TensorFlow backend.


In [28]:
#CNN architecture
numClases = 7

#training params
batch_size = 256 
num_epochs = 8 

#model parameters
num_filters = 64 
embed_dim = 300 
weight_decay = 1e-4


print("training CNN ...")
model = Sequential()
model.add(Embedding(input_dim=len(X_train), output_dim=10))
model.add(Conv1D(num_filters, 7, activation='relu', padding='same'))
model.add(MaxPooling1D(2))
model.add(Conv1D(num_filters, 7, activation='relu', padding='same'))
model.add(GlobalMaxPooling1D())
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Dense(numClases, activation='sigmoid'))  #multi-label (k-hot encoding)

adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
model.summary()

training CNN ...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, None, 10)          1595710   
_________________________________________________________________
conv1d_11 (Conv1D)           (None, None, 64)          4544      
_________________________________________________________________
max_pooling1d_6 (MaxPooling1 (None, None, 64)          0         
_________________________________________________________________
conv1d_12 (Conv1D)           (None, None, 64)          28736     
_________________________________________________________________
global_max_pooling1d_6 (Glob (None, 64)                0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 32)                2080

In [36]:
#define callbacks
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0.01, patience=0, verbose=1)
callbacks_list = [early_stopping]
#model training
hist = model.fit(X_train_bow, y_train, batch_size=batch_size, epochs=num_epochs, validation_split=0.1, shuffle=True, verbose=2)

Train on 143613 samples, validate on 15958 samples
Epoch 1/8
 - 108s - loss: 0.1635 - acc: 0.9541 - val_loss: 0.1641 - val_acc: 0.9534
Epoch 2/8
 - 122s - loss: 0.1612 - acc: 0.9541 - val_loss: 0.1612 - val_acc: 0.9534
Epoch 3/8
 - 114s - loss: 0.1596 - acc: 0.9541 - val_loss: 0.1598 - val_acc: 0.9534
Epoch 4/8
 - 118s - loss: 0.1587 - acc: 0.9541 - val_loss: 0.1587 - val_acc: 0.9534
Epoch 5/8
 - 111s - loss: 0.1585 - acc: 0.9541 - val_loss: 0.1585 - val_acc: 0.9534
Epoch 6/8
 - 112s - loss: 0.1581 - acc: 0.9541 - val_loss: 0.1578 - val_acc: 0.9534
Epoch 7/8
 - 112s - loss: 0.1578 - acc: 0.9541 - val_loss: 0.1574 - val_acc: 0.9534
Epoch 8/8
 - 111s - loss: 0.1574 - acc: 0.9541 - val_loss: 0.1578 - val_acc: 0.9534


In [37]:
y_test_pred = model.predict(X_test_bow)

In [38]:
predicted = y_test_pred
columns = ["id","toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
dfTestPredicted = pd.DataFrame(columns=columns)
for x in range(len(test)):
    dfTestPredicted.loc[x] = [test['id'][x], predicted[x][1], predicted[x][2], predicted[x][3], predicted[x][4], predicted[x][5], predicted[x][6]]
dfTestPredicted.to_csv('../reports/testPred/predTestCNN'+ str(topNFeatures) +'.csv',encoding='utf-8', index=False)